In [1]:
# %% [markdown]
# # 实验 3: 运行多种 TextAttack 攻击方法
#
# **目标:** 方便地测试不同的 TextAttack 攻击配方 (recipes) 针对 Alpaca-7B 模型的效果，可以应用不同的防御策略（无防御、AHP、SelfDenoise）。
#
# **方法:**
# 1. 设置基础环境和代理（如果需要）。
# 2. 定义一个函数 `run_single_experiment`，该函数接收攻击方法、防御方法等核心参数，并执行一次完整的攻击流程。
# 3. 在 Notebook 中定义要测试的攻击方法列表和防御方法列表。
# 4. 循环调用 `run_single_experiment` 函数来执行所有组合，或手动修改参数运行单个实验。
# 5. 加载并显示最终的结果 CSV 文件。
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["DATASETS_DISABLE_PROGRESS_BARS"] = "1"
# %% 导入基础库
import sys
import os
import pandas as pd
import subprocess
import logging
from IPython.display import display # 用于在 Notebook 中美观地显示 DataFrame
import gc
import random
# %% [markdown]
# ## 1. 环境设置与代理配置 (如果需要)

# %%
# --- 应用 AutoDL 网络加速配置 (如果在此环境) ---
logging.info("正在应用 AutoDL 网络加速配置...")
try:
    result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"',
                            shell=True, capture_output=True, text=True, check=True)
    output = result.stdout
    proxy_found = False
    for line in output.splitlines():
        if '=' in line:
            var, value = line.split('=', 1)
            value = value.strip('\'"')
            os.environ[var] = value
            logging.info(f"已设置环境变量: {var}={value}")
            proxy_found = True
    if not proxy_found:
        logging.warning("未能从 /etc/network_turbo 获取到代理环境变量。")
    else:
        logging.info("AutoDL 网络加速配置应用成功。")
except FileNotFoundError:
    logging.warning("/etc/network_turbo 文件不存在，跳过代理设置。可能不在 AutoDL 环境中。")
except subprocess.CalledProcessError as e:
    logging.error(f"执行 AutoDL 网络配置命令失败: {e}\n{e.stderr}")
except Exception as e:
    logging.error(f"应用 AutoDL 网络配置时发生未知错误: {e}")

# --- 将 src 目录添加到 Python 路径 ---
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    logging.info(f"已将 '{module_path}' 添加到 sys.path")

# --- 导入项目代码 ---
# 确保在设置代理和路径后再导入
try:
    import torch
    import transformers
    import datasets
    from src.args_config import AHPSettings
    from src.experiment_runner import ExperimentRunner
    logging.info("项目模块导入成功。")
except ImportError as e:
    logging.error(f"导入项目模块失败: {e}", exc_info=True)
    # 如果导入失败，后续代码可能无法运行

/root/ahp_env/lib/python3.12/site-packages/jieba/_compat.py:18: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
# %% [markdown]
# ## 2. 定义实验运行函数
#
# 这个函数封装了运行一次攻击实验（或评估）的逻辑。

# %%
def run_single_experiment(
    # --- 核心可变参数 ---
    attack_method: str = 'textbugger', # 要使用的 TextAttack 配方
    defense_method: str = 'none',      # 要使用的防御方法 ('none', 'ahp', 'selfdenoise')
    dataset_name: str = 'sst2',        # 数据集 ('sst2', 'agnews')
    num_examples: int = 20,           # 测试样本数量 (建议先用少量测试)

    # --- 固定或较少变动的参数 (根据需要修改默认值) ---
    mode: str = 'attack',              # 运行模式 ('attack' 或 'evaluate')
    model_path: str = '/root/autodl-tmp/circulus_alpaca-7b', # <--- !!! 请务必确认并修改为您的正确模型路径 !!!
    dataset_path: str = '/root/dataset',     # 数据集根目录
    results_file: str = '../results/experiment_results_multi_attack.csv', # 结果汇总文件
    attack_log_path: str = '../results/multi_attack_logs', # 详细攻击日志目录
    cache_dir: str = '/root/autodl-tmp/cache_path',  # 缓存目录
    model_batch_size: int = 4,         # 模型推理批次大小
    max_seq_length: int = 128,         # 最大序列长度
    mask_token: str = '<MASK>',        # 遮蔽标记
    mask_rate: float = 0.15,           # 遮蔽率
    attack_query_budget: int = 100,    # 攻击查询预算

    # --- AHP 特定参数 ---
    ahp_num_candidates: int = 10,
    ahp_pruning_method: str = 'nli',
    ahp_pruning_threshold: float = 100.0, # 困惑度阈值，越低越严格
    ahp_aggregation_strategy: str = 'majority_vote',

    # --- SelfDenoise 特定参数 ---
    selfdenoise_ensemble_size: int = 30, # 集成大小 (适当减小以加速测试)
    selfdenoise_denoiser: str = 'alpaca', # 去噪器 ('alpaca', 'roberta')

    # --- 环境参数 ---
    seed: int = 42,
    device: str = None, # 自动检测 cuda 或 cpu
    log_level: str = 'INFO',

    **kwargs # 允许传递其他未明确定义的参数 (如果 AHPSettings 支持)
):
    """
    运行单次实验（攻击或评估）。

    Args:
        (参数说明见函数定义)
    """
    logging.info(f"--- 开始实验: 数据集={dataset_name}, 防御={defense_method}, 攻击={attack_method if mode=='attack' else 'None'} ---")

    # --- 构建参数列表 ---
    # 将函数参数转换为 run_experiment.py 所需的命令行参数格式
    args_list = [
        '--mode', mode,
        '--dataset_name', dataset_name,
        '--num_examples', str(num_examples),
        '--model_path', model_path,
        '--dataset_path', dataset_path,
        '--results_file', results_file,
        '--attack_log_path', attack_log_path,
        '--cache_dir', cache_dir,
        '--model_batch_size', str(model_batch_size),
        '--max_seq_length', str(max_seq_length),
        '--mask_token', mask_token,
        '--mask_rate', str(mask_rate),
        '--defense_method', defense_method,
        '--seed', str(seed),
        '--log_level', log_level,
    ]
    # 添加设备参数 (如果指定了)
    if device:
        args_list.extend(['--device', device])

    # 根据模式添加攻击相关参数
    if mode == 'attack':
        args_list.extend([
            '--attack_method', attack_method,
            '--attack_query_budget', str(attack_query_budget),
        ])

    # 根据防御方法添加特定参数
    if defense_method == 'ahp':
        args_list.extend([
            '--ahp_num_candidates', str(ahp_num_candidates),
            '--ahp_pruning_method', ahp_pruning_method,
            '--ahp_pruning_threshold', str(ahp_pruning_threshold),
            '--ahp_aggregation_strategy', ahp_aggregation_strategy,
        ])
    elif defense_method == 'selfdenoise':
        args_list.extend([
            '--selfdenoise_ensemble_size', str(selfdenoise_ensemble_size),
            '--selfdenoise_denoiser', selfdenoise_denoiser,
        ])

    # 添加 **kwargs 中的额外参数 (如果需要)
    # for k, v in kwargs.items():
    #    args_list.extend([f'--{k}', str(v)]) # 需要确保 AHPSettings 支持这些参数

    logging.debug(f"构建的参数列表: {' '.join(args_list)}")

    # --- 解析参数并运行实验 ---
    try:
        # 1. 解析参数
        # AHPSettings().parse_args(args_list) 会设置日志、设备、种子等
        args = AHPSettings().parse_args(args_list)
        logging.info("参数解析完成，开始初始化 ExperimentRunner...")

        # 2. 初始化 Runner
        # ExperimentRunner 内部会初始化 AlpacaModel
        runner = ExperimentRunner(args)
        logging.info("ExperimentRunner 初始化完成，开始运行...")

        # 3. 运行实验 (attack 或 evaluate)
        runner.run()
        logging.info(f"--- 实验完成: 数据集={dataset_name}, 防御={defense_method}, 攻击={attack_method if mode=='attack' else 'None'} ---")
        return True # 表示成功

    except Exception as e:
        logging.error(f"--- 实验失败: 数据集={dataset_name}, 防御={defense_method}, 攻击={attack_method if mode=='attack' else 'None'} ---")
        logging.error(f"错误信息: {e}", exc_info=True) # exc_info=True 打印详细错误堆栈
        return False # 表示失败

In [5]:
# %% [markdown]
# ## 3. 定义要测试的攻击和防御组合

# %%
# --- 要测试的攻击方法列表 ---
# (基于您之前的讨论和代码中的可用性)
# 确保您已经在 src/experiment_runner.py 和 src/args_config.py 中添加了对 'bertattack' 的支持
attack_methods_to_test = [
    'textbugger',
    # 'textfooler',
    # 'pwws',
    # 'deepwordbug',
    # 'bae', # BAE 速度较慢，可以取消注释以包含它
    # 'bertattack', # 需要确保已集成
]

# --- 要测试的防御方法列表 ---
defense_methods_to_test = [
    'none',        # 无防御基线
    'ahp',         # AHP 防御
    'selfdenoise', # SelfDenoise 防御
]

# --- 要测试的数据集 ---
datasets_to_test = [
    'sst2',
    # 'agnews', # 可以取消注释以测试 AG News
]

# --- 其他通用参数 ---
# !!! 务必修改为正确的模型路径 !!!
common_params = {
    "num_examples": 20, # 设置一个合理的测试样本数量
    "model_path": '/root/autodl-tmp/circulus_alpaca-7b', # <--- !!! 再次确认路径 !!!
    "results_file": '../results/experiment_results_multi_attack.csv', # 所有结果汇总到这里
    "attack_log_path": '../results/multi_attack_logs', # 每个实验的详细日志子目录
    "mask_rate": 0.15, # 通用遮蔽率
    "attack_query_budget": 200, # 查询预算
    "ahp_pruning_method": "perplexity", # AHP 默认剪枝方法
    "ahp_aggregation_strategy": "majority_vote", # AHP 默认聚合策略
    "selfdenoise_denoiser": "alpaca", # SelfDenoise 默认去噪器
    "selfdenoise_ensemble_size": 30, # SelfDenoise 默认集成大小
    "log_level": "INFO", # 日志级别
}

# --- 清理旧的结果文件 (可选) ---
# 如果希望每次运行都生成全新的结果文件，可以在这里删除旧文件
# results_filepath = common_params["results_file"]
# if os.path.exists(results_filepath):
#     logging.warning(f"正在删除旧的结果文件: {results_filepath}")
#     os.remove(results_filepath)

In [ ]:
# %% [markdown]
# ## 4. 循环运行实验 (自动运行所有组合)
#
# **警告:** 运行所有组合可能需要很长时间！建议先使用少量样本 (`num_examples`) 和部分攻击/防御方法进行测试。

# %%
# --- 循环遍历所有组合 ---
for dataset in datasets_to_test:
    for defense in defense_methods_to_test:
        # 首先运行一次干净样本评估 (mode='evaluate', attack='None')
        logging.info(f"\n===== 开始评估: Dataset={dataset}, Defense={defense} =====")
        run_single_experiment(
            mode='evaluate', # 设置为评估模式
            attack_method='none', # 攻击方法设为 none 或其他占位符
            defense_method=defense,
            dataset_name=dataset,
            **common_params # 传递通用参数
        )

        # 然后运行所有指定的攻击方法
        for attack in attack_methods_to_test:
            # 特殊情况：如果防御是 'none' 且攻击也是 'none' (在评估中已完成)，则跳过
            if defense == 'none' and attack == 'none':
                continue

            # BAE 攻击通常非常慢，可以选择性跳过
            # if attack == 'bae':
            #     logging.warning("跳过 BAE 攻击，因为它可能非常耗时。")
            #     continue

            logging.info(f"\n===== 开始攻击: Dataset={dataset}, Defense={defense}, Attack={attack} =====")
            run_single_experiment(
                mode='attack', # 设置为攻击模式
                attack_method=attack,
                defense_method=defense,
                dataset_name=dataset,
                **common_params # 传递通用参数
            )
            # 添加一些延迟或显存清理（如果需要）
            # import time
            # time.sleep(5)
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                gc.collect()

logging.info("\n<<<<< 所有实验组合运行完毕 >>>>>")

In [8]:
# %% [markdown]
# ## 5. (可选) 手动运行单个实验
#
# 您可以复制下面的单元格，修改 `attack_method`、`defense_method` 等参数，然后单独运行该单元格来测试特定的配置。

# %%
# --- 手动运行示例 ---
run_single_experiment(
    mode='attack',
    attack_method='bertattack', # <--- 修改这里
    defense_method='none',       # <--- 修改这里
    dataset_name='sst2',        # <--- 修改这里
    # num_examples=20,            # <--- 使用少量样本测试
    **common_params
)
# # 清理显存
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
#     gc.collect()

ERROR:root:--- 实验失败: 数据集=sst2, 防御=none, 攻击=bertattack ---
ERROR:root:错误信息: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 31.36 GiB of which 159.06 MiB is free. Including non-PyTorch memory, this process has 31.19 GiB memory in use. Of the allocated memory 30.70 GiB is allocated by PyTorch, and 6.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Traceback (most recent call last):
  File "/tmp/ipykernel_3410/286068619.py", line 111, in run_single_experiment
    runner = ExperimentRunner(args)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/root/src/experiment_runner.py", line 90, in __init__
    self.alpaca_model = AlpacaModel(args)
                        ^^^^^^^^^^^^^^^^^
  File "/root/src/models/model_loader.py", line 

False

In [ ]:
# %% [markdown]
# ## 6. 加载并显示结果
#
# 加载 `results_file` 中汇总的所有实验结果。

# %%
results_filepath = common_params["results_file"]
try:
    df_results = pd.read_csv(results_filepath)
    logging.info(f"成功从 {results_filepath} 加载结果。")
    # 设置 Pandas 显示选项以查看更多内容
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1000)
    pd.set_option('display.colheader_justify', 'center')
    pd.set_option('display.precision', 4) # 显示 4 位小数

    # 显示结果 DataFrame
    display(df_results)

    # 可以进一步对结果进行排序或筛选
    # print("\n按攻击成功率降序排序:")
    # display(df_results.sort_values(by='attack_success_rate', ascending=False))

except FileNotFoundError:
    logging.error(f"结果文件 {results_filepath} 未找到。请先运行实验。")
except Exception as e:
    logging.error(f"加载或显示结果时出错: {e}")

# %% [markdown]
# --- Notebook 结束 ---